In [1]:
import h5py
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pygpe.scalar as gpe

In [2]:
# Load in data
data = h5py.File('../data/data.hdf5', 'r')
phases = data['phases']
positions = data['positions']
phases_train, phases_val = phases[:-2000, ...], phases[-2000:, ...]
positions_train, positions_val = positions[:-2000, ...], positions[-2000:, ...]

test_data = h5py.File('../data/test_data.hdf5', 'r')
phases_test = test_data['phases']
positions_test = test_data['positions']

In [3]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(filters=32, kernel_size=7, activation='relu', input_shape=(256, 256, 1)))
model.add(keras.layers.BatchNormalization(axis=3))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=3))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=3))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(4))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 250, 250, 32)      1600      
                                                                 
 batch_normalization (BatchN  (None, 250, 250, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 125, 125, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 123, 123, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 123, 123, 64)     256       
 hNormalization)                                                 
                                                        

In [4]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=1, restore_best_weights=True)

In [5]:
model.compile(optimizer='adam',loss=keras.losses.MeanSquaredError(), metrics=['accuracy'])

history = model.fit(phases_train,
                    positions_train,
                    epochs=25,
                    batch_size=8,
                    validation_data=(phases_val, positions_val),
                    callbacks=[early_stopping]
                    )

Epoch 1/25
1000/1000 [==============================] - 42s 36ms/step - loss: 52.0149 - accuracy: 0.8861 - val_loss: 15.9506 - val_accuracy: 0.9420
Epoch 2/25
1000/1000 [==============================] - 34s 34ms/step - loss: 13.3306 - accuracy: 0.9427 - val_loss: 10.6659 - val_accuracy: 0.9480
Epoch 3/25
1000/1000 [==============================] - 34s 34ms/step - loss: 12.2756 - accuracy: 0.9416 - val_loss: 13.0414 - val_accuracy: 0.9475
Epoch 4/25
1000/1000 [==============================] - 34s 34ms/step - loss: 9.0932 - accuracy: 0.9530 - val_loss: 12.4037 - val_accuracy: 0.9500
Epoch 4: early stopping


In [14]:
model.save("vortex_detection", save_format="tf")

INFO:tensorflow:Assets written to: vortex_detection\assets


INFO:tensorflow:Assets written to: vortex_detection\assets
